In [1]:
import numpy as np
import pandas as pd
import sklearn

Read from directory and creating dataframe

In [2]:
import os
array = []
for _dir in os.listdir("./textes"):
    for file in os.listdir("./textes/"+_dir):
        filepath = os.path.join("./textes/"+_dir, file)
        f = open(filepath, 'r')
        text = f.read()
        f.close()
        array.append([text,_dir])
df = pd.DataFrame(array,columns=['text','author'])
df

,text,author
0,"На единственной известной, живой планете\nНа с...",guf
1,"Давайте немного подумаем, посмеемся\nОдну секу...",guf
2,"Я вообще то не привык жаловаться на здоровье,\...",guf
3,Когда мне очень плохо или слишком хорошо\nЯ де...,guf
4,"Все, как мы любим, это просто мысли вслух\nКог...",guf
5,"Затянись мною в последний раз,\nТкни меня морд...",noizemc
6,"Если вселенная бесконечна, действительно\nЗнач...",noizemc
7,"Моё море, прошу тебя, не выплюни меня на берег...",noizemc
8,"Лица бородатые, камуфляж, автоматы,\nНеприятны...",noizemc
9,"На Марсе классно - красные пески, крутые горы ...",noizemc


SyntaxError: invalid syntax (<ipython-input-36-16d427f4b1b3>, line 1)